## 2. Servicios en ROS2
En esta sección se implementan un servidor y un cliente de servicios ROS2 usando Jupyter Notebook.
Se utilizará el servicio estándar `Trigger` y un ejecutor multihilo para permitir que ambos nodos trabajen de forma simultánea dentro del entorno interactivo.

### Importar dependencias necesarias
Se inicializan las librerías de ROS2, hilos, ejecutores y tipos de mensajes/servicios.

In [ ]:
import rclpy
from rclpy.node import Node
from std_msgs.msg import String

import threading
from rclpy.executors import MultiThreadedExecutor
from rclpy.task import Future

### Crear la clase `Server`
El nodo servidor implementa un servicio tipo `Trigger`, el cual responde con un mensaje de confirmación cada vez que recibe una solicitud.

In [2]:
from std_srvs.srv import Trigger

class Server(Node):
    def __init__(self):
        super().__init__('server_node')
        self.srv = self.create_service(
            Trigger,            # <-- tipo de servicio válido
            'mi_servicio',      # nombre del servicio
            self.callback
        )
        self.get_logger().info('Servidor listo.')

    def callback(self, request, response):
        response.success = True
        response.message = "Hola desde el servidor!"
        self.get_logger().info("Servidor recibió una solicitud.")
        return response

### Crear la clase `Client`
El cliente espera a que el servicio esté disponible y luego envía solicitudes cada segundo usando un `timer`.
La respuesta recibida se muestra en consola.


In [3]:
from std_srvs.srv import Trigger
from rclpy.task import Future

class Client(Node):
    def __init__(self):
        super().__init__('client_node')
        self.cli = self.create_client(Trigger, 'mi_servicio')

        while not self.cli.wait_for_service(timeout_sec=1.0):
            self.get_logger().info('Esperando al servidor...')

        self.timer = self.create_timer(1.0, self.timer_callback)
        self.get_logger().info('Cliente iniciado.')

    def timer_callback(self):
        req = Trigger.Request()
        future = self.cli.call_async(req)
        future.add_done_callback(self.callback_result)

    def callback_result(self, future: Future):
        try:
            resultado = future.result()
            self.get_logger().info(
                f"Respuesta: success={resultado.success}, msg='{resultado.message}'"
            )
        except Exception as e:
            self.get_logger().error(f"Error: {e}")

### Inicializar ROS2 si aún no está activo
Se utiliza `rclpy.ok()` para verificar si ROS2 ya fue inicializado dentro del entorno de Jupyter.

In [4]:
if not rclpy.ok():
    rclpy.init(args=None)

### Crear y ejecutar un `MultiThreadedExecutor`
El ejecutor permite manejar múltiples nodos al mismo tiempo.
Se ejecuta en segundo plano usando un hilo para evitar bloquear Jupyter.

In [5]:
executor = MultiThreadedExecutor()
thread = threading.Thread(target=executor.spin, daemon=True)
thread.start()

### Instanciar los nodos y agregarlos al ejecutor
Aquí se crean el servidor y el cliente, y se agregan al ejecutor.
Las salidas informativas aparecen automáticamente como evidencia del funcionamiento del servicio.

In [6]:
server_node = Server()
client_node = Client()

executor.add_node(server_node)
executor.add_node(client_node)

[INFO] [1763148367.933585942] [server_node]: Servidor listo.
[INFO] [1763148367.941533420] [client_node]: Cliente iniciado.


True

[INFO] [1763148368.941983942] [server_node]: Servidor recibió una solicitud.
[INFO] [1763148368.945365775] [client_node]: Respuesta: success=True, msg='Hola desde el servidor!'
[INFO] [1763148369.943690470] [server_node]: Servidor recibió una solicitud.
[INFO] [1763148369.950242136] [client_node]: Respuesta: success=True, msg='Hola desde el servidor!'
[INFO] [1763148370.946783897] [server_node]: Servidor recibió una solicitud.
[INFO] [1763148370.957991859] [client_node]: Respuesta: success=True, msg='Hola desde el servidor!'
[INFO] [1763148371.943248125] [server_node]: Servidor recibió una solicitud.
[INFO] [1763148371.950396768] [client_node]: Respuesta: success=True, msg='Hola desde el servidor!'
[INFO] [1763148372.948031736] [server_node]: Servidor recibió una solicitud.
[INFO] [1763148372.954526965] [client_node]: Respuesta: success=True, msg='Hola desde el servidor!'


### Destruir los nodos
Se eliminan los nodos del servidor y cliente para detener su ejecución limpia dentro del entorno.

In [7]:
server_node.destroy_node()
client_node.destroy_node()

### Apagar ROS2
Si ROS2 sigue activo, se ejecuta `rclpy.shutdown()` para evitar procesos en segundo plano.

In [8]:
if rclpy.ok():
    rclpy.shutdown()